In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum as sum_, first, round, min, when, lit, to_date, to_timestamp, date_format
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, TimestampType

# Configurações do Spark
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

# Criar sessão Spark
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Ler o CSV, já tratando "NULL" como valor nulo
TabelaCompleta = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .option('nullValue', 'NULL') \
              .csv('s3a://bucket-raw-upa-connect/tabela_historico_sensor.csv')

upa_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-raw-upa-connect/tabela_upa.csv')

unidadeDeMedida_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-raw-upa-connect/unid_medida.csv')

# Converter 'valor' para DoubleType e 'fk_upa' para IntegerType
# a = a.withColumn('valor', col('valor').cast(DoubleType())) \
#      .withColumn('fk_paciente', col('fk_paciente').cast(IntegerType()))

# TabelaCompleta.show()
# upa_df.show()
# paciente_df.show(170)
# sensor_df.show()
# unidadeDeMedida_df.show()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2cc5cc8f-20f9-4e84-91a3-25529e92509d;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 438ms :: artifacts dl 25ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

In [2]:
# Aplicar o filtro e garantir que fk_unid_medida seja inteiro
df_filtrado_inicial = TabelaCompleta.filter(
    (col("fk_sensor") == 2) & 
    (col("fk_unid_medida") == 2) & 
    (col('valor').isNotNull())
).withColumn(
    'valor', col('valor').cast(DoubleType())
).withColumn(
    'fk_unid_medida', col('fk_unid_medida').cast(IntegerType())
).withColumnRenamed(
    'valor', 'umidade'
)

# Exibir
df_filtrado_inicial.show()

+-------------------+-------+------+-----------+---------+--------------+
|          data_hora|umidade|fk_upa|fk_paciente|fk_sensor|fk_unid_medida|
+-------------------+-------+------+-----------+---------+--------------+
|2025-05-20 00:00:00|  89.53|    33|       null|        2|             2|
|2025-05-20 00:00:00|  -6.13|    32|       null|        2|             2|
|2025-05-20 00:00:00|  84.49|    31|       null|        2|             2|
|2025-05-20 00:00:00|  82.54|    30|       null|        2|             2|
|2025-05-20 00:00:00|  55.49|    29|       null|        2|             2|
|2025-05-20 00:00:00|  72.41|    28|       null|        2|             2|
|2025-05-20 00:00:00|  33.11|    27|       null|        2|             2|
|2025-05-20 00:00:00|  77.76|    26|       null|        2|             2|
|2025-05-20 00:00:00|  63.99|    34|       null|        2|             2|
|2025-05-20 00:00:00|  37.22|    25|       null|        2|             2|
|2025-05-20 00:00:00|  32.92|    15|  

In [3]:
# Tirando outliers
tirar_outliers_umidade = (col('fk_sensor') == 2) & (col('valor') >= 0) & (col('valor') <= 100)

df_filtrado_final = df_filtrado_inicial.filter(
    tirar_outliers_umidade
).drop('fk_paciente')  # <-- remove a coluna fk_paciente

# Mostrar o DataFrame filtrado
df_filtrado_final.show()

# Salvar no bucket trusted
df_filtrado_final.coalesce(1) \
    .write \
    .option('header', 'true') \
    .mode('overwrite') \
    .csv('s3a://bucket-trusted-upa-connect/tabela_umidade_tratada.csv')

+-------------------+-------+------+---------+--------------+
|          data_hora|umidade|fk_upa|fk_sensor|fk_unid_medida|
+-------------------+-------+------+---------+--------------+
|2025-05-20 00:00:00|  89.53|    33|        2|             2|
|2025-05-20 00:00:00|  84.49|    31|        2|             2|
|2025-05-20 00:00:00|  82.54|    30|        2|             2|
|2025-05-20 00:00:00|  55.49|    29|        2|             2|
|2025-05-20 00:00:00|  72.41|    28|        2|             2|
|2025-05-20 00:00:00|  33.11|    27|        2|             2|
|2025-05-20 00:00:00|  77.76|    26|        2|             2|
|2025-05-20 00:00:00|  63.99|    34|        2|             2|
|2025-05-20 00:00:00|  37.22|    25|        2|             2|
|2025-05-20 00:00:00|  32.92|    15|        2|             2|
|2025-05-20 00:00:00|  71.38|    14|        2|             2|
|2025-05-20 00:00:00|  47.91|    13|        2|             2|
|2025-05-20 00:00:00|  72.01|    12|        2|             2|
|2025-05

25/05/31 22:19:32 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/05/31 22:19:32 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [4]:
df_tabela_filtrada_trusted = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .option('nullValue', 'NULL') \
              .csv('s3a://bucket-trusted-upa-connect/tabela_umidade_tratada.csv')

In [5]:
from pyspark.sql.functions import date_trunc, avg, round, col

# Join com unidade de medida
df_com_unidade = df_tabela_filtrada_trusted.join(
    unidadeDeMedida_df.select(
        col('id_unidade_de_medida').alias('fk_unid_medida'),
        col('unidade_de_medida')
    ),
    on='fk_unid_medida',
    how='left'
)

# Truncar para a hora
df_por_hora = df_com_unidade.withColumn(
    'data_hora', date_trunc('hour', col('data_hora'))
)

# Calcular média
media_por_hora = df_por_hora.groupBy(
    'fk_upa', 'data_hora', 'unidade_de_medida'
).agg(
    round(avg('umidade'), 2).alias('media_umid_hora')
)

# ✔️ Selecionar na ordem desejada
df_final = media_por_hora.select(
    'fk_upa', 'data_hora', 'media_umid_hora', 'unidade_de_medida'
).orderBy('fk_upa', 'data_hora')

# Mostrar
df_final.show(truncate=False)


+------+-------------------+---------------+-----------------+
|fk_upa|data_hora          |media_umid_hora|unidade_de_medida|
+------+-------------------+---------------+-----------------+
|1     |2025-05-20 00:00:00|53.51          |%                |
|1     |2025-05-20 01:00:00|52.52          |%                |
|1     |2025-05-20 02:00:00|58.42          |%                |
|1     |2025-05-20 03:00:00|58.24          |%                |
|1     |2025-05-20 04:00:00|49.89          |%                |
|1     |2025-05-20 05:00:00|56.83          |%                |
|1     |2025-05-20 06:00:00|59.82          |%                |
|1     |2025-05-20 07:00:00|54.36          |%                |
|1     |2025-05-20 08:00:00|56.14          |%                |
|1     |2025-05-20 09:00:00|59.13          |%                |
|1     |2025-05-20 10:00:00|51.47          |%                |
|1     |2025-05-20 11:00:00|43.17          |%                |
|1     |2025-05-20 12:00:00|51.19          |%          

In [6]:
# Fazendo o join primeiro
df_final = df_final.join(
    upa_df.select(
        col('id_upa').alias('fk_upa'), 
        col('nome').alias('nome_da_upa')
    ),
    on='fk_upa',
    how='left'
)

# Ordenando
df_final = df_final.orderBy('fk_upa', 'data_hora')

# Criando as colunas de data e hora
df_final = df_final.withColumn('data', to_date(col('data_hora'), 'yyyy-MM-dd HH:mm:ss').cast(DateType())) \
                    .withColumn('hora', date_format(col('data_hora'), 'HH:mm:ss').cast(StringType())) \
                    .drop('data_hora')

# Adicionando a coluna legenda
df_final = df_final.withColumn(
    'legenda',
    when(col('media_umid_hora') > 60, 'Umidade ACIMA do limite')
    .when(col('media_umid_hora') < 40, 'Umidade ABAIXO do limite')
    .otherwise('Umidade dentro do limite')
)

# Reordenando as colunas
df_final = df_final.select(
    'fk_upa',
    'data',
    'hora',
    'nome_da_upa',
    'media_umid_hora',
    'unidade_de_medida',
    'legenda'
)

# Exibindo
df_final.show(truncate=False)

+------+----------+--------+---------------+---------------+-----------------+------------------------+
|fk_upa|data      |hora    |nome_da_upa    |media_umid_hora|unidade_de_medida|legenda                 |
+------+----------+--------+---------------+---------------+-----------------+------------------------+
|1     |2025-05-20|00:00:00|UPA 21 DE JUNHO|53.51          |%                |Umidade dentro do limite|
|1     |2025-05-20|01:00:00|UPA 21 DE JUNHO|52.52          |%                |Umidade dentro do limite|
|1     |2025-05-20|02:00:00|UPA 21 DE JUNHO|58.42          |%                |Umidade dentro do limite|
|1     |2025-05-20|03:00:00|UPA 21 DE JUNHO|58.24          |%                |Umidade dentro do limite|
|1     |2025-05-20|04:00:00|UPA 21 DE JUNHO|49.89          |%                |Umidade dentro do limite|
|1     |2025-05-20|05:00:00|UPA 21 DE JUNHO|56.83          |%                |Umidade dentro do limite|
|1     |2025-05-20|06:00:00|UPA 21 DE JUNHO|59.82          |%   

In [7]:
df_final.coalesce(1) \
        .write \
        .option('header', 'true') \
        .mode('overwrite') \
        .csv('s3a://bucket-client-upa-connect/tabela_umidade_pronta.csv')

25/05/31 22:19:45 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/05/31 22:19:46 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [8]:
%pip install gspread google-auth-oauthlib boto3 pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip uninstall gspread -y
!pip install gspread==3.7.0

Found existing installation: gspread 6.0.0
Uninstalling gspread-6.0.0:
  Successfully uninstalled gspread-6.0.0


In [10]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [11]:
# --- Instalação de bibliotecas (adicione esta seção no início do seu script) ---
import subprocess
import sys

# Função para instalar pacotes
def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"'{package}' instalado com sucesso.")
    except Exception as e:
        print(f"Erro ao instalar '{package}': {e}")
        # Em ambientes de produção, você pode querer adicionar um exit(1) aqui
        # para parar a execução se uma dependência crítica não puder ser instalada.

# Instalar as bibliotecas necessárias
install_package("pandas")
install_package("gspread")
install_package("google-auth-oauthlib") # Necessário para google.oauth2.service_account.Credentials
install_package("boto3") # Para interagir com o S3
# --- Fim da seção de instalação de bibliotecas ---


# --- CÓDIGO PARA ENVIAR PARA O GOOGLE SHEETS ---
import pandas as pd # <-- MOVIDO PARA AQUI PARA GARANTIR QUE 'pd' ESTEJA NO ESCOPO
import gspread
from google.oauth2.service_account import Credentials
import boto3
import os
import sys

# === CONFIGURAÇÕES PARA O GOOGLE SHEETS ===
GOOGLE_SHEET_ID = '1i6BfuZXPOcTp6BFAiVkpktOBym0HtakZacUKfDzu1zI'
S3_BUCKET_CREDENTIALS = 'bucket-client-upa-connect'
S3_KEY_CREDENTIALS = 'credenciais.json'
LOCAL_CREDENTIALS_PATH = '/tmp/credenciais.json'

# === AUTENTICAÇÃO GOOGLE ===
print(f"Baixando credenciais do S3: s3://{S3_BUCKET_CREDENTIALS}/{S3_KEY_CREDENTIALS} para {LOCAL_CREDENTIALS_PATH}...")
s3 = boto3.client('s3')
try:
    s3.download_file(S3_BUCKET_CREDENTIALS, S3_KEY_CREDENTIALS, LOCAL_CREDENTIALS_PATH)
    print("Credenciais baixadas com sucesso.")
except Exception as e:
    print(f"Erro ao baixar credenciais do S3: {e}")
    sys.exit(1)

scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
credenciais = Credentials.from_service_account_file(LOCAL_CREDENTIALS_PATH, scopes=scopes)
cliente = gspread.authorize(credenciais)
planilha = cliente.open_by_key(GOOGLE_SHEET_ID)

# === ENVIA O DATAFRAME FINAL PARA A NOVA ABA "umidade" ===
NOME_ABA_TESTE = "UmidadeAmbiente"
print(f"Enviando DataFrame final para a aba '{NOME_ABA_TESTE}'...")

# Converter Spark DataFrame para Pandas DataFrame
try:
    df_pandas = df_final.toPandas()
    print("DataFrame Spark convertido para Pandas com sucesso.")
except Exception as e:
    print(f"Erro ao converter Spark DataFrame para Pandas: {e}")
    print("O DataFrame pode ser muito grande para a memória do driver ou a sessão Spark está inválida.")
    sys.exit(1)

# --- NOVO TRECHO: Ajustar tipagem das colunas para o Google Sheets ---
# Apenas formata colunas de data/hora para string ISO 8601.
# As demais colunas (numéricas, strings, etc.) são mantidas em seus tipos nativos no Pandas.
# O gspread e o Google Sheets farão a inferência de tipo para elas.

# # Coluna 'data_chegada': assegurar que é string DD/MM/YYYY
if 'data' in df_pandas.columns:
    if pd.api.types.is_datetime64_any_dtype(df_pandas['data']):
        df_pandas['data'] = df_pandas['data'].dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    elif pd.api.types.is_string_dtype(df_pandas['data']):
        # Tentar converter strings que podem não estar em formato padrão para datetime,
        # e depois formatar para DD/MM/YYYY. 'errors=coerce' transforma não-datas em NaT.
        df_pandas['data'] = pd.to_datetime(df_pandas['data'], errors='coerce').dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    # Preencher valores nulos (NaT) com string vazia, pois None também pode causar problemas.
    df_pandas['data'] = df_pandas['data'].fillna('')

# # Coluna 'horario_chegada': assegurar que é string HH:MM:SS
if 'hora' in df_pandas.columns:
    if pd.api.types.is_datetime64_any_dtype(df_pandas['hora']):
        df_pandas['hora'] = df_pandas['hora'].dt.strftime('%H:%M:%S')
    elif pd.api.types.is_string_dtype(df_pandas['hora']):
        # Se já é string (como no seu caso vindo do Spark), apenas garanta que nulos sejam preenchidos.
        df_pandas['hora'] = df_pandas['hora'].fillna('')


# Nenhuma conversão global df_pandas.astype(str) é necessária aqui.
# gspread é capaz de lidar com int, float, bool, e strings diretamente.

dados = [df_pandas.columns.tolist()] + df_pandas.values.tolist()


try:
    aba = planilha.worksheet(NOME_ABA_TESTE)
    aba.clear()
    print(f"Aba '{NOME_ABA_TESTE}' encontrada e limpa.")
except gspread.exceptions.WorksheetNotFound:
    aba = planilha.add_worksheet(title=NOME_ABA_TESTE, rows=str(len(dados) + 100), cols=str(len(df_pandas.columns) + 10))
    print(f"Aba '{NOME_ABA_TESTE}' criada.")

try:
    aba.update('A1', dados)
    print(f"Dados do DataFrame final enviados com sucesso para a aba '{NOME_ABA_TESTE}'.")
except Exception as e:
    print(f"Erro ao atualizar a aba do Google Sheets: {e}")

# Opcional: Remover o arquivo de credenciais temporário
if os.path.exists(LOCAL_CREDENTIALS_PATH):
    os.remove(LOCAL_CREDENTIALS_PATH)
    print(f"Arquivo de credenciais temporário '{LOCAL_CREDENTIALS_PATH}' removido.")

# Parar a sessão Spark
# spark.stop()

'pandas' instalado com sucesso.


'gspread' instalado com sucesso.


'google-auth-oauthlib' instalado com sucesso.


'boto3' instalado com sucesso.
Baixando credenciais do S3: s3://bucket-client-upa-connect/credenciais.json para /tmp/credenciais.json...


/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Credenciais baixadas com sucesso.
Enviando DataFrame final para a aba 'UmidadeAmbiente'...


DataFrame Spark convertido para Pandas com sucesso.
Aba 'UmidadeAmbiente' encontrada e limpa.
Dados do DataFrame final enviados com sucesso para a aba 'UmidadeAmbiente'.
Arquivo de credenciais temporário '/tmp/credenciais.json' removido.
